In [68]:
#Loading the required packages
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from collections import OrderedDict
from collections import deque
import re

In [2]:
#Changing the working directory
os.chdir('E:/Capstone IATH/Code')

In [ ]:
#Attach code for getting these urls here

In [72]:
#URLs for the collection pages
links = ['http://cbw.iath.virginia.edu/books_display.php?id=1319',
'http://cbw.iath.virginia.edu/books_display.php?id=1325',
'http://cbw.iath.virginia.edu/books_display.php?id=1339',
'http://cbw.iath.virginia.edu/books_display.php?id=1354',
'http://cbw.iath.virginia.edu/books_display.php?id=1374',
'http://cbw.iath.virginia.edu/books_display.php?id=1467',
'http://cbw.iath.virginia.edu/books_display.php?id=1484',
'http://cbw.iath.virginia.edu/books_display.php?id=1486',
'http://cbw.iath.virginia.edu/books_display.php?id=1511',
'http://cbw.iath.virginia.edu/books_display.php?id=1512',
'http://cbw.iath.virginia.edu/books_display.php?id=1534',
'http://cbw.iath.virginia.edu/books_display.php?id=1548',
'http://cbw.iath.virginia.edu/books_display.php?id=1549',
'http://cbw.iath.virginia.edu/books_display.php?id=1598',
'http://cbw.iath.virginia.edu/books_display.php?id=1611',
'http://cbw.iath.virginia.edu/books_display.php?id=1632',
'http://cbw.iath.virginia.edu/books_display.php?id=1650',
'http://cbw.iath.virginia.edu/books_display.php?id=1745',
'http://cbw.iath.virginia.edu/books_display.php?id=1803',
'http://cbw.iath.virginia.edu/books_display.php?id=1864',
'http://cbw.iath.virginia.edu/books_display.php?id=1894',
'http://cbw.iath.virginia.edu/books_display.php?id=1968',
'http://cbw.iath.virginia.edu/books_display.php?id=1970',
'http://cbw.iath.virginia.edu/books_display.php?id=2006',
'http://cbw.iath.virginia.edu/books_display.php?id=2024',
'http://cbw.iath.virginia.edu/books_display.php?id=2058',
'http://cbw.iath.virginia.edu/books_display.php?id=2077',
'http://cbw.iath.virginia.edu/books_display.php?id=2086',
'http://cbw.iath.virginia.edu/books_display.php?id=2136',
'http://cbw.iath.virginia.edu/books_display.php?id=2137',
'http://cbw.iath.virginia.edu/books_display.php?id=2141',
'http://cbw.iath.virginia.edu/books_display.php?id=2161',
'http://cbw.iath.virginia.edu/books_display.php?id=2212',
'http://cbw.iath.virginia.edu/books_display.php?id=2236',
'http://cbw.iath.virginia.edu/books_display.php?id=2243',
'http://cbw.iath.virginia.edu/books_display.php?id=2269',
'http://cbw.iath.virginia.edu/books_display.php?id=2395',
'http://cbw.iath.virginia.edu/books_display.php?id=2439',
'http://cbw.iath.virginia.edu/books_display.php?id=2446']

In [74]:
#Collecting the urls of biographies from the collection urls using selenium
allurls = []
for link in links:
    browser = webdriver.Chrome()
    browser.get(link)
    buttons = browser.find_elements_by_xpath('//button[text()="Text"]')
    for button in buttons:
        button.click()
    urls = []
    for handle in browser.window_handles:
        browser.switch_to.window(handle)
        urls.append(browser.current_url)
        browser.close()
    urls = deque(urls)
    urls.popleft()
    allurls.extend(urls)


In [77]:
#List of urls of the biographies
allurls

['http://cbw.iath.virginia.edu/exist/cbw/basic/a001/bio70',
 'http://cbw.iath.virginia.edu/exist/cbw/basic/a001/bio20',
 'http://cbw.iath.virginia.edu/exist/cbw/basic/a001/bio04',
 'http://cbw.iath.virginia.edu/exist/cbw/basic/a007/bio13',
 'http://cbw.iath.virginia.edu/exist/cbw/basic/a007/bio12',
 'http://cbw.iath.virginia.edu/exist/cbw/basic/a020/bio10',
 'http://cbw.iath.virginia.edu/exist/cbw/basic/a020/bio09',
 'http://cbw.iath.virginia.edu/exist/cbw/basic/a020/bio07',
 'http://cbw.iath.virginia.edu/exist/cbw/basic/a020/bio06',
 'http://cbw.iath.virginia.edu/exist/cbw/basic/a020/bio05',
 'http://cbw.iath.virginia.edu/exist/cbw/basic/a020/bio04',
 'http://cbw.iath.virginia.edu/exist/cbw/basic/a020/bio03',
 'http://cbw.iath.virginia.edu/exist/cbw/basic/a020/bio02',
 'http://cbw.iath.virginia.edu/exist/cbw/basic/a020/bio01',
 'http://cbw.iath.virginia.edu/exist/cbw/basic/a034/bio12',
 'http://cbw.iath.virginia.edu/exist/cbw/basic/a034/bio11',
 'http://cbw.iath.virginia.edu/exist/cbw

In [79]:
# Open File
result = open("listofurls.csv",'w')

# Write data to file
for url in allurls:
    result.write(url + "\n")
result.close()

In [92]:
#Creating an empty dataframe for getting the text of all the biographies
data = pd.DataFrame()

In [93]:
#Getting the content
for url in allurls:
    html = requests.get(url)
    soup = BeautifulSoup(html.text,'lxml')
    pno = soup.find_all(class_ = 'PNum')
    pt = soup.find_all(class_ = 'text')
    parano = [p.text for p in pno]
    text = [t.text for t in pt]
    col = [('ParagraphNo', parano), ('ParagraphText', text)]
    df = pd.DataFrame.from_dict(OrderedDict(col))
    df['CollectionID'] = url.split('/')[-2]
    df['BiographyID'] = url.split('/')[-1]
    data = data.append(df)

In [97]:
#Reset index for the dataframe
data.reset_index(drop=True, inplace=True)

In [119]:
#Remove the paragraph number from the text column using the following function
def removepno(text):
    pos = 0
    for i,x in enumerate(text):
        if x.isalpha():        
            pos = i                  
            break
    return text[pos:]

In [120]:
data['ParagraphText'] = data['ParagraphText'].apply(removepno)

In [122]:
#Cleaning the text column to remove duplicate spaces, newline characters tab characters, leading and trailing spaces
text = data['ParagraphText']
text = [t.replace("\n","").strip() for t in text]
text = [t.replace("\t","") for t in text]
text = [re.sub(' +', ' ', t) for t in text]
data['ParagraphText'] = text

In [124]:
#Writing to a csv file
data.to_csv("textdatafromweb.csv", index = False)